# Initial conditions
1Hz pacing for 1000 seconds

In [ ]:
using ProgressLogging
using OrdinaryDiffEq
using ModelingToolkit
using ECMEDox
using ECMEDox: second, mM, Hz, μM, mV
using Plots
using DisplayAs: PNG

tend = 1000.0second
bcl = 1.0second
@named sys = build_model()
u0 = build_u0(sys)
sts = unknowns(sys)
alg = KenCarp47()
@unpack iStim = sys
callback = build_stim_callbacks(iStim, tend; period=bcl)
prob = ODEProblem(sys, u0, tend)

In [ ]:
@time sol = solve(prob, alg; reltol=1e-6, abstol=1e-6, progress=true, callback = callback)

for i in sts
    istr = replace(string(i), "(t)" => "")
    println("sys.", istr, " => ", sol[i][end], ",")
end

In [ ]:
plot(sol, idxs=sys.vm, legend=:right, tspan=(900second, 901second)) |> PNG

Citric acid cycle metabolites
Citrate and isocitrate have the highest concentrations

In [ ]:
@unpack cit, isoc, oaa, akg, scoa, suc, fum, mal = sys
plot(sol, idxs=[cit, isoc, oaa, akg, scoa, suc, fum, mal], legend=:right, title="CAC metabolites") |> PNG

CAC flux

In [ ]:
@unpack vMDH, vAAT, vIDH = sys
plot(sol, idxs=[vMDH, vAAT, vIDH], legend=:right, title="CAC flux") |> PNG

Q cycle

In [ ]:
@unpack Q_n, SQn, QH2_n, QH2_p, Q_p, SQp, fes_ox, fes_rd, cytc_ox, cytc_rd = sys
plot(sol, idxs=[Q_n + Q_p, SQn, QH2_n + QH2_p, SQp], title="Q cycle", legend=:left, xlabel="Time (ms)", ylabel="Conc. (μM)") |> PNG

Q cycle downstream

In [ ]:
plot(sol, idxs=[fes_ox, fes_rd, cytc_ox, cytc_rd], title="Q cycle (downstream)", legend=:left, xlabel="Time (ms)", ylabel="Conc. (μM)") |> PNG

In [ ]:
plot(sol, idxs = [sys.vHresC1, sys.vHresC3, sys.vHresC4], ylims=(0, 3)) |> PNG

In [ ]:
plot(sol, idxs = [sys.sox_i, sys.sox_m], tspan=(900e3, 910e3)) |> PNG

ROS

In [ ]:
plot(sol, idxs = [sys.vROSIf, sys.vROSIq, sys.vROSC1, sys.vROSC3], tspan=(900e3, 910e3)) |> PNG

O2 Shunt

In [ ]:
plot(sol, idxs=100 * sys.vROS / (sys.vO2 + sys.vROS), title="O2 Shunt", tspan=(900e3, 910e3)) |> PNG

MMP

In [ ]:
plot(sol, idxs = [sys.dpsi], tspan=(900e3, 910e3)) |> PNG

ATP synthesis rate

In [ ]:
plot(sol, idxs = [sys.vC5], tspan=(900e3, 910e3)) |> PNG

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*